In [8]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [9]:
def parse(x):
    return pd.datetime.strptime(x, '%Y %m %d %H')
df = pd.read_csv('weather.csv',  parse_dates = {'date':[1,2,3,4]} ,index_col='No', date_parser=parse,skiprows=0)

df=df.fillna(df.mean()) ###df.fillna(method='bfill') 用后面填充，pad 前面填充
#df.isnull().values.any()   false

In [10]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1] #8
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1): ##i=1
		cols.append(df.shift(i)) #8  下移一步
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]#i=1
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):#i=0
		cols.append(df.shift(-i)) ###上移0步
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [71]:
values = df.values
encoder = LabelEncoder()
values[:,5] = encoder.fit_transform(values[:,5])
########去除时间戳
#values1 = values[:,1:]

# ensure all data is float
#values1 = values1.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values[:,1:])
scaled = scaled.astype('float32')
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
#print(reframed[120:140])

E:\Anaconda\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [74]:
reframed.to_csv('formatweather.csv')